In [ ]:
import requests
import io
import os
import time

import numpy as np
import sys
import tensorflow as tf

#librerías para graficar
import matplotlib.pyplot as plt
                 
plt.rcParams['figure.figsize'] = (16, 9)  #ver graficas grandes 
plt.style.use('ggplot')
#guardar las imagenes y tablas en el cuaderno de jupyter
%matplotlib inline 

In [22]:
print("Version: {}\n Eager mode: {} ".format(tf.__version__,tf.executing_eagerly))
print("GPU is","available" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")

Version: 2.7.0
 Eager mode: <function executing_eagerly at 0x7f2de6845a70> 
GPU is available


In [23]:
tf.device('/cpu:0')

#Descarga y procesamiento de datos

In [25]:
fileUrl = 'https://raw.githubusercontent.com/2HenryCardenas1/DeepLearning/main/Jupyter/PLN(Procesamiento_de_lenguajes_naturales)/Datasets/la_biblioteca_de_babel.txt'
fileDL= tf.keras.utils.get_file('la_biblioteca_de_babel.txt',fileUrl)
texto = open(fileDL, 'rb').read().decode(encoding='utf-8')

##Normalizamos el texto

In [26]:
print("[!] Pasando todo el texto a minuscula")
texto_min = texto.lower()
time.sleep(1)
print("[!] Quitando los saltos de linea del texto")
texto_sin_salto = texto_min.replace('\n', '')
texto_sin_r = texto_sin_salto.replace('\r','\n')
texto_final = texto_sin_r.replace('\n', '')
time.sleep(1)
print("Hecho !!")

[!] Pasando todo el texto a minuscula
[!] Quitando los saltos de linea del texto
Hecho !!


In [27]:
print('el texto tiene longitud de:{} caracteres'. format(len(texto_final)))
vocab = sorted(set(texto_final))
print('el texto esta compuesto de estos :{} caracteres'. format(len(texto_final)))
print(vocab)
print(texto_final)

el texto tiene longitud de:15688 caracteres
el texto esta compuesto de estos :15688 caracteres
[' ', '!', '(', ')', ',', '-', '.', '1', '7', ':', ';', '?', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'x', 'y', 'z', '¡', '«', '»', '¿', 'ñ']
el universo (que otros llaman la biblioteca) se compone de un numero indefinido, ytal vez infinito, de galerias hexagonales, con vastos pozos de ventilacion en el medio,cercados por barandas bajisimas. desde cualquier hexagono se ven los pisos inferiores ysuperiores: interminablemente. la distribucion de las galerias es invariable. veinteanaqueles, a cinco largos anaqueles por lado, cubren todos los lados menos dos; su altura,que es la de los pisos, excede apenas la de un bibliotecario normal. una de las caras libresda a un angosto zaguan, que desemboca en otra galeria, identica a la primera y a todas. aizquierda y a derecha del zaguan hay dos gabinetes minusculos. uno permite dormir depie;

##Pasamos el texto a numeros


In [28]:
chars = sorted(list(set(texto_final)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
print(chars)
print(char_to_int,"\n")

n_chars = len(texto_final)
n_vocab = len(chars)
print("En total hay %d caracteres, y el diccionario tiene un tamaño de %d caracteres." % (n_chars, n_vocab))

[' ', '!', '(', ')', ',', '-', '.', '1', '7', ':', ';', '?', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'x', 'y', 'z', '¡', '«', '»', '¿', 'ñ']
{' ': 0, '!': 1, '(': 2, ')': 3, ',': 4, '-': 5, '.': 6, '1': 7, '7': 8, ':': 9, ';': 10, '?': 11, 'a': 12, 'b': 13, 'c': 14, 'd': 15, 'e': 16, 'f': 17, 'g': 18, 'h': 19, 'i': 20, 'j': 21, 'l': 22, 'm': 23, 'n': 24, 'o': 25, 'p': 26, 'q': 27, 'r': 28, 's': 29, 't': 30, 'u': 31, 'v': 32, 'x': 33, 'y': 34, 'z': 35, '¡': 36, '«': 37, '»': 38, '¿': 39, 'ñ': 40} 

En total hay 15688 caracteres, y el diccionario tiene un tamaño de 41 caracteres.


###Dividimos el texto en secuencias:
---
Dividimos el texto en estas secuencias, convertimos los caracteres a números enteros usando nuestra tabla de búsqueda que preparamos anteriormente

In [29]:
# preparar el conjunto de datos de los pares de entrada a salida codificados como enteros
seq_length = 50
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
	seq_in = texto_final[i:i + seq_length]
	seq_out = texto_final[i + seq_length]
	dataX.append([char_to_int[char] for char in seq_in])
	dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print ("Se generaron {} secuencias de texto de un tamaño de {} caracteres". format(n_patterns,seq_length))

Se generaron 15638 secuencias de texto de un tamaño de 50 caracteres


#Entrenamiento

##Preparamos nuestros datos de entrenamiento

---


1.   Primero debemos transformar la lista de secuencias de entrada en la forma [muestras, pasos de tiempo, características] esperada por una red LSTM.
2.   Luego debemos cambiar la escala de los números enteros al rango de 0 a 1 para que los patrones sean más fáciles de aprender mediante la red LSTM que utiliza la función de activación sigmoidea de forma predeterminada.
3.   por ultimo necesitamos convertir los patrones de salida (caracteres individuales convertidos en enteros) en una codificación one hot. Esto es para que podamos configurar la red para predecir la probabilidad de cada uno de los 54 caracteres diferentes en el vocabulario (una representación más fácil)


In [30]:
#transformar la lista X de secuencias de entrada en la forma [muestras, pasos de tiempo, características]
X = np.reshape(dataX, (n_patterns, seq_length, 1))
# normalizar (cambiar la escala de los números enteros al rango de 0 a 1 )
X = X / float(n_vocab)
# convertir los patrones de salida (caracteres individuales convertidos en enteros) en una codificación one hot.
y = tf.keras.utils.to_categorical(dataY)

##Construcción del modelo RNN

---
definimos nuestro modelo LSTM: 
Aquí definimos una única capa LSTM oculta con 256 unidades de memoria. La red usa deserción con una probabilidad de 20. La capa de salida es una capa densa que usa la función de activación softmax para generar una predicción de probabilidad para cada uno de los 54 caracteres entre 0 y 1.

In [33]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.LSTM(256, input_shape=(X.shape[1], X.shape[2])))    #creamos una capa con 256 unidades de memoria
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(y.shape[1], activation='softmax'))            #Softmax convierte un vector de valores en una distribución de probabilidad para cada uno de los 39
                                                                              #Softmax se utiliza a menudo como la activación para la última capa de una red de clasificación
#utilizamos el algoritmo de optimización de ADAM para la velocidad
model.compile(loss='categorical_crossentropy', optimizer='adam')

###Creando chekpoints

---

La red es lenta de entrenar (alrededor de 300 segundos por época) teniendo activa la GPU, ASí que crearemos CHECKPOINTS (puntos de control) para registrar todos los pesos de la red para archivar cada vez que se observe una mejora en la pérdida al final de la época. Usaremos el mejor conjunto de pesos (menor pérdida) para instanciar nuestro modelo generativo en la siguiente sección

In [34]:
# definimos  los checkpoint
filepath="CheckPoints/weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath, monitor='loss', verbose=0, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

###Entrenando

In [ ]:
import os
while True: 
  for i in range(100):
    os.remove('/content/drive/MyDrive/CheckPoints/weights-improvement-{}'.format(i))

In [35]:
history=model.fit(X, y, epochs=1000, batch_size=128, callbacks=callbacks_list, verbose=True)

Epoch 1/1000
123/123 [==============================] - 4s 21ms/step - loss: 2.9842
Epoch 2/1000
123/123 [==============================] - 2s 20ms/step - loss: 2.9189
Epoch 3/1000
123/123 [==============================] - 2s 20ms/step - loss: 2.9095
Epoch 4/1000
123/123 [==============================] - 2s 20ms/step - loss: 2.9023
Epoch 5/1000
123/123 [==============================] - 2s 20ms/step - loss: 2.8963
Epoch 6/1000
123/123 [==============================] - 2s 20ms/step - loss: 2.8832
Epoch 7/1000
123/123 [==============================] - 2s 20ms/step - loss: 2.8465
Epoch 8/1000
123/123 [==============================] - 2s 20ms/step - loss: 2.8117
Epoch 9/1000
123/123 [==============================] - 2s 20ms/step - loss: 2.7928
Epoch 10/1000
123/123 [==============================] - 2s 20ms/step - loss: 2.7768
Epoch 11/1000
123/123 [==============================] - 2s 20ms/step - loss: 2.7574
Epoch 12/1000
123/123 [==============================] - 2s 20ms/step - lo

In [ ]:
to_restore = tf.Variable(callbacks_list)
print(to_restore.numpy())  # Puros ceros
fake_layer = tf.train.Checkpoint(bias=to_restore)
fake_net = tf.train.Checkpoint(l1=fake_layer)
new_root = tf.train.Checkpoint(net=fake_net)
status = new_root.restore(tf.train.latest_checkpoint('/content/drive/MyDrive/CheckPoints/'))
new = to_restore.numpy()
print(new)  # Ahora obtenemos el valor restaurado

In [ ]:
status.assert_existing_objects_matched()

##Generando texto con una red LSTM


---
Vamos a cargar el ultimo CHECKPOINT de entrenamiento y con el haremos MAGIA!!!


In [36]:
filename = "/content/CheckPoints/weights-improvement-984-0.0222.hdf5" #Aca va el checkpoint con mejor resultado (el ultimo :)
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

###Mapeo inverso (números a letras)
creamos un mapeo inverso que podamos usar para convertir los números enteros nuevamente en caracteres para que podamos entender las predicciones

In [37]:
int_to_char = dict((i, c) for i, c in enumerate(chars))

##Predicciones

In [38]:
# elige una semilla al azar
start = np.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print ("Semilla:")
print ("\"" + " ".join([int_to_char[value] for value in pattern])+"\"")
# generación de caracteres
for i in range(1000):
	x = np.reshape(pattern, (1, len(pattern), 1))
	x = x / float(n_vocab)
	prediction = model.predict(x, verbose=0)
	index = np.argmax(prediction)
	result = int_to_char[index]
	seq_in = [int_to_char[value] for value in pattern]
	sys.stdout.write(result)
	pattern.append(index)
	pattern = pattern[1:len(pattern)]
print ("\nDone.")

Semilla:
"e   l e   d i j o   q u e   e s t a b a n   r e d a c t a d a s   e n   p o r t u g u e s ;   o t r"
os ledijeron que en yiddish. antes de un siglo pudo establecerse el idioma: un dialectosamoyedo-lituano del guarani, con inflexiones de arabe clasico. tambien se descifro elcontenido: nociones de analisis combinatorio, ilustradas por ejemplos de variaciones conrepeticion ilimitada. esos ejemplos permitieron que un bibliotecario de genio descubrierala ley fundamental de la biblioteca. este pensador observo que todos los libros, pordiversos que sean, constan de elementos iguales: el espacio, el punto, la coma, lasveintidos letras del alfabeto. tambien alego un hecho que todos los viajeros hanconfirmado: no hay en la vasta biblioteca, dos libros identicos. de esas premisasincontrovertibles dedujo que la biblioteca es total y que sus anaqueles registran todas lasposibles combinaciones de los veintitantos simbolos ortograficos (numero, aunquevastisimo, no infinito) o sea todo lo qu

##Mejorando la red


In [39]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(tf.keras.layers.Dropout(0.2))
#pero agregaremos una segunda capa.
model.add(tf.keras.layers.LSTM(256))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
#cambiamos el nombre de archivo de los pesos con puntos de control para que 
#podamos distinguir entre los pesos de esta red 
filepath="CheckPoints/weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"

###Mejoramos el entrenamiento

---
aumentamos las epoch y disminuiremos el tamaño del lote de 128 a 64 para darle a la red más oportunidades de actualizarse y aprender.

In [40]:
#los tiempos de entrenamiento aumentaran al doble que en la versión anterior
model.fit(X, y, epochs=50, batch_size=64, callbacks=callbacks_list)

Epoch 1/50
245/245 [==============================] - 12s 36ms/step - loss: 2.9531
Epoch 2/50
245/245 [==============================] - 9s 35ms/step - loss: 2.9147
Epoch 3/50
245/245 [==============================] - 9s 35ms/step - loss: 2.8679
Epoch 4/50
245/245 [==============================] - 9s 35ms/step - loss: 2.7842
Epoch 5/50
245/245 [==============================] - 9s 35ms/step - loss: 2.7360
Epoch 6/50
245/245 [==============================] - 9s 35ms/step - loss: 2.6994
Epoch 7/50
245/245 [==============================] - 9s 35ms/step - loss: 2.6724
Epoch 8/50
245/245 [==============================] - 8s 35ms/step - loss: 2.6397
Epoch 9/50
245/245 [==============================] - 9s 35ms/step - loss: 2.6095
Epoch 10/50
245/245 [==============================] - 9s 35ms/step - loss: 2.5744
Epoch 11/50
245/245 [==============================] - 9s 35ms/step - loss: 2.5373
Epoch 12/50
245/245 [==============================] - 9s 35ms/step - loss: 2.4949
Epoch 13/50


###Haciendo predicciones

In [41]:
# elige una semilla al azar
start = np.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print ("Semilla:")
print ("\"" + " ".join([int_to_char[value] for value in pattern])+"\"")
# generación de caracteres
for i in range(1000):
	x = np.reshape(pattern, (1, len(pattern), 1))
	x = x / float(n_vocab)
	prediction = model.predict(x, verbose=0)
	index = np.argmax(prediction)
	result = int_to_char[index]
	seq_in = [int_to_char[value] for value in pattern]
	sys.stdout.write(result)
	pattern.append(index)
	pattern = pattern[1:len(pattern)]
print ("\nDone.")

Semilla:
"l e d i j e r o n   q u e   e n   y i d d i s h .   a n t e s   d e   u n   s i g l o   p u d o   e"
stablecerse el idioma panodadan los actos decada hombre del universo y guardaban arcanos prodigiosos para su porvenir. mires decodiciosos abaada aloune ee la biblioteca. este pensador observo que todos los hibros y besan con barbarie las paginas, pero nosaben descifrar una sola letra. las epidemias, las discordias hereticas, las de esscaher de la paginas  hon dsar, la bimo. las paligiddas. es teraad cuadeechos. el caba labra ee comsendr, de eomeriaraste tenuaion de la tuya, es computable en cero.tambien se espero entonces la aclaracion de los mistesios basicos de la humanidad: elorigen de la biblioteca y del tiempo. es verosido la delaralis de los mistesios basicos de la humanidad: elorigen de la biblioteca y del tiempo. es verosido la delaralis de los mistesios basicos de la humanidad: elorigen de la biblioteca y del tiempo. es verosido la delaralis de los mistesios basicos d

##Exportar modelo RNN

In [42]:
!pip install h5py

In [44]:
from keras.models import model_from_json
import os
# Serializamos el modelo en forma JSON
model_json = model.to_json()
with open("modelRNN_cuentos.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("/content/modeloRNN_cuentosPesos.hdf5")
model.save('modelRNN_cuentos_v_h5.h5')
print("modelo salvado en disco")

modelo salvado en disco


###Cargando un modelo

In [46]:
# Recrea exactamente el mismo modelo solo desde el archivo
new_model = tf.keras.models.load_model('/content/modelRNN_cuentos_v_h5.h5')

In [48]:
chars = sorted(list(set("comiendo una manzana")))
char_to_int = dict((c, i) for i, c in enumerate(chars))
n_chars = len(texto_final)
n_vocab = len(chars)
print("En total hay %d caracteres, y el diccionario tiene un tamaño de %d caracteres." % (n_chars, n_vocab))
pattern = dataX[5]
print(pattern)

En total hay 15688 caracteres, y el diccionario tiene un tamaño de 11 caracteres.
[20, 32, 16, 28, 29, 25, 0, 2, 27, 31, 16, 0, 25, 30, 28, 25, 29, 0, 22, 22, 12, 23, 12, 24, 0, 22, 12, 0, 13, 20, 13, 22, 20, 25, 30, 16, 14, 12, 3, 0, 29, 16, 0, 14, 25, 23, 26, 25, 24, 16]


In [49]:
start = np.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print ("Semilla:")
print ("\"" + " ".join([int_to_char[value] for value in pattern])+"\"")
# generación de caracteres
for i in range(1000):
	x = np.reshape(pattern, (1, len(pattern), 1))
	x = x / float(n_vocab)
	prediction = new_model.predict(x, verbose=0)
	index = np.argmax(prediction)
	result = int_to_char[index]
	seq_in = [int_to_char[value] for value in pattern]
	sys.stdout.write(result)
	pattern.append(index)
	pattern = pattern[1:len(pattern)]
print ("\nDone.")

Semilla:
"r a   m i ,   q u e   s e a n   p a r a   o t r o s .   q u e   e l   c i e l o   e x i s t a ,   a"
oeaaaaaaaaaaaa, ioeaaaaaaaaaa, ioeaaaaaaaaaaa, moeaaaaaaaaaaaaaaaaaaaaaaaaaaa, moeaaaa, ioeaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa, moeaaa,,mo, m soaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa q qaaarrrr, ioeaaa, m sasaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa q qaaarrrr, ioeaaa, m sasaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa q qaaarrrr, ioeaaa, m sasaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa q qaaarrrr, ioeaaa, m sasaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa q qaaarrrr, ioeaaa, m sasaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa q qaaarrrr, ioeaaa, m sasaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa q qaaarrrr, ioeaaa, m sasaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa q qaaarrrr, ioeaaa, m sasaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa q qaaarrrr, ioeaaa, m sasaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa q qaaarrrr, ioeaaa, m sasaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa q qaaarrrr,